In [1]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader
import tqdm
tqdm.tqdm()
import nltk
nltk.download('punkt')


0it [00:00, ?it/s]
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alealcoforado/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:

# Define your sentence transformer model using CLS pooling
model_name = 'sentence-transformers/stsb-xlm-r-multilingual'
# word_embedding_model = models.Transformer(model_name)
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer(model_name)

# Define a list with sentences (1k - 100k sentences)
train_data = pd.Series(["Eu também. Acho que não. Sei que 123.45 é igual.",
                   "Your set of sentences",
                   "Model will automatically add the noise", 
                   "And re-construct it",
                   "You should provide at least 1k sentences"])


In [5]:
import datasets_handler
import pandas as pd
train_sentences = datasets_handler.splitDocuments(pd.Series(train_data))

In [6]:
# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True) # decoder_name_or_path=model_name, 

Some weights of XLMRobertaForCausalLM were not initialized from the model checkpoint at /Users/alealcoforado/.cache/torch/sentence_transformers/sentence-transformers_stsb-xlm-r-multilingual/ and are newly initialized: ['encoder.layer.0.crossattention.output.dense.weight', 'encoder.layer.3.crossattention.output.LayerNorm.weight', 'encoder.layer.1.crossattention.output.dense.bias', 'encoder.layer.5.crossattention.output.LayerNorm.weight', 'encoder.layer.11.crossattention.self.query.weight', 'encoder.layer.0.crossattention.self.value.bias', 'encoder.layer.8.crossattention.self.value.weight', 'encoder.layer.9.crossattention.output.LayerNorm.bias', 'encoder.layer.11.crossattention.output.LayerNorm.bias', 'encoder.layer.0.crossattention.output.LayerNorm.bias', 'encoder.layer.6.crossattention.self.key.bias', 'encoder.layer.7.crossattention.output.dense.bias', 'encoder.layer.10.crossattention.self.value.bias', 'encoder.layer.9.crossattention.self.value.bias', 'encoder.layer.5.crossattention.se

In [7]:
# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=10,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
# model.save('/Users/alealcoforado/Documents/Projetos/Modelos/sentence-transformers/roberta-test-1')